<a href="https://colab.research.google.com/github/juancuevas-ops/analisis-molecular-covid19/blob/main/Skin_disease_detector_Fann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# prompt: quiero instalar tensorflow para usar keras model y layer intalalo

!pip install tensorflow
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
print(tf.__version__)
print("TensorFlow installation successful")


2.18.0
TensorFlow installation successful


In [16]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.layers import DepthwiseConv2D
from tensorflow.keras.models import Model
from PIL import Image

# Función personalizada para DepthwiseConv2D
def custom_depthwise_conv2d(**kwargs):
    kwargs.pop('groups', None)
    return DepthwiseConv2D(**kwargs)

custom_objects = {'DepthwiseConv2D': custom_depthwise_conv2d}

# Ruta del modelo
path_to_model = "/content/modelo_jeysshon_iaderm.h5"

# Cargar el modelo con custom_objects
try:
    model = tf.keras.models.load_model(path_to_model, custom_objects=custom_objects)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
    raise

# Definir las etiquetas
labels = [
    'Acné / Rosácea', 'Queratosis Actínica / Carcinoma Basocelular',
    'Dermatitis Atópica', 'Enfermedad Bullosa',
    'Celulitis Impétigo (Infecciones Bacterianas)',
    'Eczema', 'Exanthems (Erupciones Cutáneas por Medicamentos)', 'Pérdida de Cabello (Alopecia)',
    'Herpes HPV', 'Trastornos de Pigmentación',
    'Lupus',
    'Melanoma (Cáncer de Piel)', 'Hongos en las Uñas',
    'Hiedra Venenosa',
    'Psoriasis (Lichen Planus)', 'Sarna Lyme',
    'Queratosis Seborreica', 'Enfermedad Sistémica',
    'Tinea Ringworm (Infecciones Fúngicas)',
    'Urticaria Ronchas', 'Tumores Vasculares', 'Vasculitis', 'Verrugas Molusco'
]

# Función Grad-CAM para generar el mapa de calor de activación
def grad_cam(model, img_array, last_conv_layer_name="top_conv", pred_index=None):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Generar el mapa de calor de activación usando Grad-CAM
def generate_heatmap(image, last_conv_layer_name="top_conv"):
    img_array = tf.image.resize(image, (224, 224))
    img_array = tf.expand_dims(img_array, axis=0)
    heatmap = grad_cam(model, img_array, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_image = cv2.addWeighted(np.array(image), 0.6, heatmap, 0.4, 0)
    return Image.fromarray(superimposed_image)

# Función principal para clasificar la imagen y generar el mapa de calor
def classify_image(image):
    # Redimensionar y preparar la imagen para predicción
    image_resized = tf.image.resize(image, (224, 224))
    image_resized = tf.expand_dims(image_resized, axis=0)
    prediction = model.predict(image_resized).flatten()
    confidences = {labels[i]: float(prediction[i]) for i in range(len(labels))}

    # Generar mapa de calor
    heatmap_image = generate_heatmap(np.array(image))
    return confidences, heatmap_image

# Configuración de la interfaz de Gradio
title = "AI-DERM DETECTION: Diagnóstico Automatizado de Enfermedades Cutáneas con Mapa de Calor"

article = (
    "Se propone un sistema automatizado para el diagnóstico de las 23 enfermedades comunes de la piel:\n\n"
    "1. Acné / Rosácea\n"
    "2. Queratosis Actínica / Carcinoma Basocelular\n"
    "3. Dermatitis Atópica\n"
    "4. Enfermedades Bullosas\n"
    "5. Celulitis / Impétigo (Infecciones Bacterianas)\n"
    "6. Eccema\n"
    "7. Exantemas (Erupciones Cutáneas por Medicamentos)\n"
    "8. Pérdida de Cabello (Alopecia)\n"
    "9. Herpes / VPH\n"
    "10. Trastornos de la Pigmentación\n"
    "11. Lupus\n"
    "12. Melanoma (Cáncer de Piel)\n"
    "13. Hongos en las Uñas\n"
    "14. Hiedra Venenosa\n"
    "15. Psoriasis (liquen plano)\n"
    "16. Sarna / Enfermedad de Lyme\n"
    "17. Queratosis Seborreica\n"
    "18. Enfermedad Sistémica\n"
    "19. Tiña / Tiña (Infecciones Fúngicas)\n"
    "20. Urticaria / Ronchas\n"
    "21. Tumores Vasculares\n"
    "22. Vasculitis\n"
    "23. Verrugas / Molusco\n\n"
    "Este sistema automatizado se basa en un modelo preentrenado EfficientNetB7, capaz de diagnosticar 23 enfermedades cutáneas comunes. La interfaz te permite cargar una imagen y obtener las probabilidades de cada enfermedad detectada.\n\n"
    "Además, el sistema incorpora Grad-CAM, una técnica de visualización que genera un mapa de calor superpuesto a la imagen de entrada, "
    "destacando las áreas clave que el modelo considera relevantes para el diagnóstico. Esto permite a los profesionales de la salud y usuarios "
    "comprender mejor el razonamiento del modelo al realizar una predicción, facilitando la interpretación y confianza en el diagnóstico automatizado.\n\n"
    "<p style='text-align: center'>"
    "<span style='font-size: 15pt;'>AI-DERM . Jeysshon Bustos . 2023.</span>"
    "</p>"
)

description = (
    "AI-DERM DETECTION es una herramienta avanzada que permite cargar imágenes de afecciones cutáneas para su diagnóstico automático. "
    "El modelo clasifica la imagen en una de las 23 categorías de enfermedades cutáneas más comunes. Además, gracias a la implementación de Grad-CAM, "
    "se muestra un mapa de calor superpuesto sobre la imagen, resaltando las zonas donde el modelo ha encontrado patrones que contribuyen a la clasificación. "
    "Esta visualización permite entender mejor el diagnóstico y proporciona información adicional sobre las áreas afectadas en la piel."
)

examples = [
    ['./123.jpg'],
    ['./acne-closed-comedo-2.jpg'],
    ['./distal-subungual-onychomycosis-86.jpeg'],
    ['./cherry-angioma-16.jpg'],
    ['./malignant-melanoma-16.jpg'],
    ['./tinea-primary-lesion-15.jpeg'],
    ['./congenital-nevus-35.jpg'],
    ['./tinea-body-137.jpg'],
    ['./atopic-13.jpg'],
    ['./atopic-7.jpg']
]

# Lanzar la interfaz de Gradio
gr.Interface(
    fn=classify_image,
    title=title,
    article=article,
    description=description,
    inputs=gr.Image(),
    outputs=[gr.Label(num_top_classes=4), gr.Image(label="Mapa de Calor")],
    examples=examples
).launch()

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Modelo cargado exitosamente.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d22a5f75d00181e9a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
